In [207]:
## Import packages
import numpy as np
import math
import random
from scipy.stats import poisson
import time

## Setup problem definition

## Test case

- T = 4h
- d = 5 min # the size of an interval
- beta = 9 minutes # service time (model by the exponential distribution)
- N = 24 patients
- I = T / d # the number of intervals

In [208]:
class schedule:
  def __init__(self, beta: int, I: int, d: int, N: int):
    self.beta = beta # = 1/μ : average service time
    self.I = I # number of intervals
    self.d = d # length of interval
    self.N = N # total number of patients
    self.x:list[int] = np.repeat(0, self.I) # schedule with x[t] as number of 
                                            # patients scheduled at the start of
                                            # interval t, t = 1,...,T; initially
                                            # set at zero
                                            
  def reset_schedule(self):
    for t in range(self.I):
      self.x[t] = 0

  def make_random_schedule(self, min_x: int, max_x: int, step: int=1):
    n = self.N
    for i in range(0,self.I, step):
      if n >= 0:
        r = random.randint(min_x, max_x)
        self.x[i] = min(r,n)
        n = n - r
        print(n, self.x[i])
      else:
        return
        
  def make_initial_schedule(self):
    for i in range(self.N):
      t = round(i*self.I / self.N)
      if i > self.I:
        i-= 1
      self.x[t] += 1

## Functions

In [209]:
# moving patients around iteratively 
# local search
def simpleSearch(x, beta, precision, limit, v, n, no_show, I, d, alpha_W, alpha_I, alpha_T, results):
  for m in range(0, I-1):
    # for k in list(range(0, I))[::-1]: # [::-1]
    k = I-1
    while k >= 0:
      x_current = x.copy()
      if x_current[k] > 0: # Adding one vector is equivalent to moving the arrival of one patient from interval k to interval k+1
        next_k = (k + m + 1) % I

        x_current[k] -= 1
        x_current[next_k] += 1

        temp_results = calcResults(x_current, beta, precision, limit, v, n, no_show, I, d, 0, alpha_W, alpha_I, alpha_T)
        
        if temp_results['objVal'] < results['objVal']:
          x = x_current
          results = temp_results
          k += 1

        else: # undo the previous move
          x_current[k] += 1
          x_current[next_k] -= 1
          k -= 1
        
      else:
        k -= 1
    
  return x, results

In [210]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999): # Poisson distribution 
  k = 0
  p = []

  while sum(p) < precision: # fill accurate values up to precision limit
    p.append(poisson.pmf(k, beta))
    k+=1

  while len(p) < size: # fill the rest of the values with 0
    p.append(0)
  return p, k

In [211]:
def calcExponentialLimit(mu):
  return int(max(mu+4*mu**0.5,100))

In [212]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))

def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [213]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k] # I+1
  return tardiness


def calcIdletime(I, d, tardiness, n, no_show, beta):
  return (I * d) + tardiness - (n * (1 - no_show) * beta) # I-1?


def calcWaitingtime(p_min, x, p, limit, I, n):
  w = np.zeros((I+1, n+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k] 
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= n
  return waitingtime

In [214]:
def calculate_v(p, beta, precision, precision_limit, n, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*n) + 1
  v = np.zeros((n+1, limit+d))
  add_v = np.zeros((n+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, n+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i] 
      i += 1

  for k in range(n+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1
          
  return v, limit

In [215]:

def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1,limit+d+1))
  p_min = np.zeros((I+1,limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1): # calculate p_min and p_plus iteratively 
    # Constraint 3
    for k in range(d+1):
      p_min[t][0] += p_plus[t-1][k]

    # Constraint 4
    for i in range(1,limit+1):
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I: # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

In [216]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess

In [217]:
def calcResults(x, beta, precision, limit, v, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, n, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, n)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness
  
  print(f"Schedule: {x},\nObjective value: {objVal},\nProb calculation time: {probT} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT} sec),\nIdle time (timer): {idletime} ({idletimeT} sec),\nTardiness (timer): {tardiness} ({tardT} sec)\n")

  # Collect into a dictionary
  results = {'p_min' : p_min, 'waitingTime' : waitingtime, 'idleTime' : idletime, 'tardiness' : tardiness, 'objVal' : objVal}
  
  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess
  
  return results


# Main Function

In [218]:
precision = 0.9999
# n = 50 # number of patients
n = 24
beta = 9 # average service time for a patient
T = 4*60 # total time
d = 5 # interval size
I = int(T/d) # number of intervals

no_show = 0
iend = 0

s = schedule(beta,I,d,n)
s.make_initial_schedule()
# x = s.x
x = list(np.zeros(I, dtype=int)) # alternate x with all patients at the beginning
x[0] = n

size = calcExponentialLimit(beta*n)+1 # size of p has to be at least as big as the limit value here
p, precision_limit = calculate_p(beta, size, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, n, d, no_show)

alpha_I = 0.2
alpha_T = 0.4 # patient doctor centric slider
alpha_W = 0.4

# results = calcResults(x, beta, precision, limit, v, n, no_show, I, d, 0, alpha_W, alpha_I, alpha_T)

# bestObjVal = results['objVal']
x, results = simpleSearch(x, beta, precision, limit, v, n, no_show, I, d, alpha_W, alpha_I, alpha_T, results)
# while results['objVal'] < bestObjVal:
#   bestObjVal = results['objVal']
#   x, results = simpleSearch(x, beta, precision, limit, v, n, no_show, I, d, alpha_W, alpha_I, alpha_T, results)

Schedule: [23, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
Objective value: 46.26666712764462,
Prob calculation time: 2.0043516049991013 sec,
Waiting time (timer): 103.16846587730623 (1.1519774430016696 sec),
Idle time (timer): 24.332134627870204 (1.8560021999292076e-06 sec),
Tardiness (timer): 0.33213462787020853 (0.00015204599912976846 sec)

Schedule: [23, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
Objective value: 46.183507274143565,
Prob calculation time: 2.0112138410004263 sec,
Waiting time (timer): 102.96053456632457 (1.1398954830001458 sec),
Idle time (timer): 24.33215574602289 (2.420998498564586e-06 sec),
Tardiness (timer): 0.33215574602289205 (0.0001564979975228198 sec)

Schedule: [23, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

KeyboardInterrupt: 

# A problem is introduced when the precision is not enough for p_min when there are few patients in a large schedule

# TODO schedule metrics bugged with certain schedules -> p_min not calced properly? or issue with precision?

# TODO search can be finished but still has to perform I**2 number of probability calculations, introduce sane limit for objval?

# TODO optimal schedule is not found after one iteration of simpleSearch, efficiency of first patients moved depend on position of earlier patients in the list

# is it quicker to move everything per iteration or abandon an iteration if something is moved?